<a href="https://colab.research.google.com/github/JaneOliveira/ML-Perceptron/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# brincando com ML

####Criando Perceptron
---
Sem usar conceitos de orientação a objetos

In [ ]:

import numpy as np

def cria_base_dados():

    base_dados = np.genfromtxt('base.txt')

    return base_dados

def cria_conj_teste(base, n_amostras, coluna):
    
    conj_teste = np.zeros((n_amostras, coluna))
    
    for i in range(n_amostras):
        for j in range(coluna):
            conj_teste[i,j] = base[i,j]
       
    return conj_teste

def deleta_test(base, n_amostras):
    for i in range(n_amostras):
        base = np.delete(base,i, 0)
    
    return base;

def funcao_ativacao(mu):
    if mu>=0:
        return 1
    else:
        return -1
    
def neuronio(x,w):
    mu = np.dot(x.T,w)
    y_hat = funcao_ativacao(mu)
    return y_hat

def perceptron(dados, eta=0.1, max_iter = 2000, erro_tol = 0.001):
    n = dados.shape[0]  #numero de linhas
    d = dados.shape[1] #numero de colunas
    x = np.concatenate((np.ones((n,1)), dados[:,0:-1]), axis=1)
    y = dados[:,-1]  #view altera no local de origem
    
    w = np.random.randn(d)
    
    it = 0
    erro_total = erro_tol + 10;
    
    while((it<max_iter) and (erro_total> erro_tol)):
        erro_total = 0
        for i in range(n):
            #calcular saida do neuronio
            y_hat = neuronio(x[i,:],w)
            #verificar o erro
            erro = y[i] - y_hat
            #atualizar os pesos
            dw = eta*erro*x[i,:]
            w += dw
            erro_total += np.abs(erro)
            
        it +=1
    
    return w, erro_total, it

def normalizacao(dados):
    
    norma = dados/dados.max(axis=0)
    
    return norma
    

def previsao_perceptron( teste, w):
    resultado=np.zeros((teste.shape[0]))
    
    x = np.concatenate((np.ones((teste.shape[0],1)), teste[:,0:-1]), axis=1)
    for i in range(teste.shape[0]):
            #calcular saida do neuronio
            y_hat = neuronio(x[i,:],w)
            resultado[i] = y_hat
    
    return resultado

def compara_resultado(resultado, teste):
    acertos = 0
    erros = 0
    for i in range(teste.shape[0]):
        if(resultado[i] == (teste[i,-1])):
            acertos +=1
        else:
            erros+=1
    
    return erros, acertos
       

def main():
    
    base = cria_base_dados()
    base[base[:,-1] == 2, -1] = -1 
    
    base = normalizacao(base)
    
    n_amostras = int((base.shape[0])*0.2)
    coluna = int((base.shape[1]))
    
    conj_teste = cria_conj_teste(base, n_amostras, coluna)
    
    base = deleta_test(base, n_amostras)
    
    w, erro, iteracoes = perceptron(base, eta=0.7, max_iter = 1000, erro_tol = 0.001)
    
    resultado = previsao_perceptron(conj_teste, w)
    
    erros, acertos = compara_resultado(resultado, conj_teste)
    

    perc_acertos = acertos*100/(len(conj_teste))
    perc_erros = erros*100/(len(conj_teste))
    print("--------------------------")
    print("Resultados classificação perceptron")
    print("")
    print("--------------------------\n")
    print(f"Acertos:{acertos} de um total de {len(conj_teste)} amostras \nEquivale a {perc_acertos}%")
    print(f"\nErros:{erros} de um total de {len(conj_teste)} amostras \nEquivale a {perc_erros}%")

    print("--------------------------")

main()

--------------------------
Resultados classificação perceptron

--------------------------

Acertos:149 de um total de 200 amostras 
Equivale a 74.5%

Erros:51 de um total de 200 amostras 
Equivale a 25.5%
--------------------------


####Criando Perceptron
---
Com conceitos de orientação a objetos

In [ ]:
import numpy as np

class Perceptron:
    def __init__(self, data, learning_rate=0.7, n_iterations=2000)-> None:
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.dataset = data
        self.n, self.d = np.shape(self.dataset)

    def cria_conj_test(self) -> None:
      self.n_amostras = int((self.dataset.shape[0])*0.2)
      coluna = int((self.dataset.shape[1]))
      np.random.shuffle(self.dataset)
      self.conjunto_teste = np.zeros((self.n_amostras, coluna))

      for i in range(self.n_amostras):
          for j in range(coluna):
              self.conjunto_teste[i,j] = self.dataset[i,j]
      self.deleta_teste()

    
    def deleta_teste(self) ->None:
      for i in range(self.n_amostras):
        self.dataset = np.delete(self.dataset,i, 0)


    def normaliza(self) -> None:
      nInput = self.d - 1
      for l in range(nInput):
        self.dataset[:,l] = 2*(self.dataset[:,l] - self.dataset[:,l].min(axis=0)) / (self.dataset[:,l].max(axis=0) - self.dataset[:,l].min(axis=0)) - 1

    def fit(self)-> None:
        X = self.dataset[:,:-1]
        X[np.isnan(X)] = 0
        y = self.dataset[:,-1]
        n_samples, n_features = self.dataset[:,:-1].shape


        # Inicialize os pesos e o bias com zeros
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Treine o modelo por um número fixo de iterações
        for i in range(self.n_iterations):
            for j in range(n_samples):
                # Calcule a saída do modelo
                output = self.predict(X[j])

                # Atualize os pesos e o bias usando a regra do Perceptron
                try:              
                  self.weights += self.learning_rate * (y[j] - output) * X[j]
                  self.bias += self.learning_rate * (y[j] - output)
                except:
                  self.weights += 0.000000001
                  self.bias += 0.0000000001
    
    def activation(self, x)-> int:
        # Use a função de ativação degrau
        return 1 if x >= 0 else -1

    def predict(self, X)-> int:

        # Calcule a saída do modelo
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation(linear_output)

        return y_predicted

    def result(self)-> None:
        self.acertos =0
        self.erros =0
        # Calcula resultado conjunto teste
        for i in range(self.conjunto_teste.shape[0]):
          y_predicted = self.predict(self.conjunto_teste[i,:-1])

          if(y_predicted == self.conjunto_teste[i,-1]):
            self.acertos+=1
          else:
            self.erros+=1

        self.por_acertos = self.acertos*100/(len(self.conjunto_teste))
        self.por_erros = self.erros*100/(len(self.conjunto_teste))
        print("--------------------------")
        print("Resultados classificação Perceptron")
        print("")
        print("--------------------------\n")
        print(f"Acertos:{self.acertos} de um total de {len(self.conjunto_teste)} amostras \nequivale a {self.por_acertos}%")
        print(f"Erros:{self.erros} de um total de {len(self.conjunto_teste)} amostras \nequivale a {self.por_erros}%")

        print("--------------------------")



def main():
  dataset = np.genfromtxt('base.txt')
  dataset[dataset[:,-1] == 2, -1] = -1  

  model = Perceptron(dataset)
  model.normaliza()
  model.cria_conj_test()
  model.fit()
  model.result()
main()

--------------------------
Resultados classificação Perceptron

--------------------------

Acertos:142 de um total de 200 amostras 
equivale a 71.0%
Erros:58 de um total de 200 amostras 
equivale a 29.0%
--------------------------
